In [1]:
import pandas as pd 
import jenkspy
import geopandas as gpd 
import numpy as np 
import os 
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt 
#%matplotlib inline 
#from functools import partial
from pyproj import Proj, transform 

path = os.getcwd()
rootpath= 'c:\\Users\\Gamelab\\Desktop\\RT\\Resilience_Lab\\MRA_COMPILED_2\\population_density\\'

# 1. Residential Density 1996 

Process to calculate: 
- get the main residential buildings file (which was clipped from the main osm file, taking 11 residential categories)
- then clip these residential buildings to the residential polygons of the 1996 landuse file (do this for every year)
- now load the populations shapefile of 1996, which contains, the buurt and the inwoners (population of that neighborhood)
- run the 'join by location' , with geometric predicate: 'within' and 'contains'. add the following columns to the buildings file: 'onderwerpe','gemeente','inwoners' 
- read the created shapefile below. remove unecessary columns. group the buildings by buurt and sum the areas to get a dictionary of 'buurt' and the 'residential area' of that buurt
- similarly, group the buurts by population and create a buurt and population dictionary. 
- to an empty dataframe, add the buurt, the residential area of buurt, the population of the buurt 
- ensure you don't have missing values 
- calculate the area in two ways: sq meters and sq kilomteres

- perform fisher-jenks to derive the classifcations of the residential density column. 

In [4]:
## get all the building areas shapefile. groupby the buurt and sum up area1 for all buildings within that buurt. 
#buurt: BU_NAAM, population: AANT_INW, area: area1 
#drop: fclass, code, fid, layer, path, name, res_area
#buildingareas = buildingareas.drop(columns=['path','res_area','Onderwerpe','Gemeentena'])
#buildingareas.to_file(path+'\\buildingareas_buurts1997_1.shp')

In [5]:
## get all the building areas shapefile. groupby the buurt and sum up area1 for all buildings within that buurt. 
buildingareas = gpd.read_file(rootpath+'buildingareas_1996_buurts.shp')
#buurt: BU_NAAM, population: AANT_INW, area: area1 
#drop: fclass, code, fid, layer, path, name, res_area

buildingareas = buildingareas.drop(columns=['fid','code','fclass','name','layer','Inwoners'])
buildingareas = buildingareas.rename(columns={'Onderwerpe':'buurt','Gemeentena':'Gemeente','area1':'area','Inwoners_2':'Inwoners'})

buildingareas = buildingareas.dropna(subset=['buurt'])
buildingareas.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 496490 entries, 0 to 496489
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   osm_id    496490 non-null  object  
 1   type      496490 non-null  object  
 2   area      496490 non-null  int64   
 3   buurt     496490 non-null  object  
 4   Gemeente  496490 non-null  object  
 5   Inwoners  496490 non-null  float64 
 6   geometry  496490 non-null  geometry
dtypes: float64(1), geometry(1), int64(1), object(4)
memory usage: 30.3+ MB


In [7]:
buildingareas['buurt'] = buildingareas['buurt'].astype('string')
buildingareas['area'] = buildingareas['area'].astype('int')
buildingareas['Inwoners']=buildingareas['Inwoners'].astype('int')

totalareas = buildingareas.groupby('buurt')['area'].sum()
buurtpopulations = buildingareas.groupby('buurt')['Inwoners'].median().to_dict() 


#creating the dataframe we are going to store this in 
areadf = pd.DataFrame()
areadf['buurt'] = list(totalareas.keys())  ## add the buurt name 
areadf['residentialarea']= list(totalareas.values)  ## add the calculated residential area per buurt 

pops = []
for buurt in list(areadf['buurt']):
    pops.append(buurtpopulations[buurt])
areadf['population'] = pops    ## adding the population of the buurt 
## instead of 

density = []
density_sqm= []

for _,row in areadf.iterrows():
    density_sqm.append((row['population']/row['residentialarea']))
    density.append((row['population']/row['residentialarea'])*0.000001)  #residential density in km square

areadf['residentialdensity_sqkm'] = density 
areadf['residential_sqm']= density_sqm
## now add buurt geometries to areadf
buurts1997 = gpd.read_file(path+'\\mra_populationdensity_1997.shp') 
geomap = {}
buurts1997['Onderwerpe'] = buurts1997['Onderwerpe'].astype('string')
for _,row in buurts1997.iterrows():
    geomap[row['Onderwerpe']] = row['geometry']

areadf['geometry'] = areadf['buurt'].map(geomap)
areagdf = GeoDataFrame(areadf, crs="EPSG:28992", geometry=list(areadf['geometry']))
areagdf.head()


,buurt,residentialarea,population,residentialdensity_sqkm,residential_sqm,geometry
0,'s-Graveland,24796,1230,4.960477e-08,0.049605,"POLYGON ((137875.406 474664.812, 137907.328 47..."
1,'t Hoogt van 't Kruis,57655,850,1.474287e-08,0.014743,"POLYGON ((140412.094 469255.812, 140788.797 46..."
2,'t Spil,7877,260,3.300749e-08,0.033007,"POLYGON ((131114.406 496077.594, 131557.094 49..."
3,Aalsmeerderbrug,23500,530,2.255319e-08,0.022553,"POLYGON ((113074.797 477238.594, 113177.148 47..."
4,Abbenes,39800,1010,2.537688e-08,0.025377,"POLYGON ((100000.000 472367.688, 99831.039 472..."


In [26]:
#export 
#areagdf.to_file(path+'\\residential_density_calculated_1996.shp')
areagdf= gpd.read_file(path+'\\residential_density_calculated_1996.shp')
areagdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   buurt       739 non-null    object  
 1   residentia  739 non-null    int64   
 2   population  739 non-null    int64   
 3   resident_1  739 non-null    float64 
 4   resident_2  739 non-null    float64 
 5   geometry    739 non-null    geometry
dtypes: float64(2), geometry(1), int64(2), object(1)
memory usage: 34.8+ KB


# Outliers

In [27]:
print(len(areagdf))
print('Number of outliers:', len(areagdf.loc[areagdf['resident_2']>=3]))
areagdf.loc[areagdf['resident_2']>=3]

739
Number of outliers: 5


,buurt,residentia,population,resident_1,resident_2,geometry
136,De Omval,32,430,0.000013,13.437500,"POLYGON ((123653.297 483636.000, 124707.219 48..."
357,Meer en Dorp,86,4340,0.000050,50.465116,"MULTIPOLYGON (((100860.046 478147.261, 100863...."
410,Nigtevecht,41,1010,0.000025,24.634146,"MULTIPOLYGON (((130168.109 476163.203, 130190...."
620,Verspr. h. in het oosten,4,120,0.000030,30.000000,"MULTIPOLYGON (((101238.837 479277.134, 101467...."
658,Vreeland,82,1580,0.000019,19.268293,"MULTIPOLYGON (((131088.610 470831.879, 131078...."


In [28]:
#areagdf = gpd.read_file(path+'\\residential_density_calculated_1996.shp')
areagdf_without_outliers = areagdf.loc[areagdf['resident_2']<=3]
print(len(areagdf_without_outliers))
areagdf_without_outliers.sort_values(by='resident_1',ascending=False)


734


,buurt,residentia,population,resident_1,resident_2,geometry
220,Hilversumse Meent landelijk gebied,18,40,2.222222e-06,2.222222,"POLYGON ((135744.953 477545.688, 135908.547 47..."
257,IJ-eiland e.o.,108,220,2.037037e-06,2.037037,"MULTIPOLYGON (((126447.395 488087.616, 126138...."
229,Hoofddorp e.o.,47284,56130,1.187082e-06,1.187082,"POLYGON ((110960.203 483659.188, 110722.602 48..."
592,Tussenbekenbuurt,1819,1860,1.022540e-06,1.022540,"POLYGON ((103689.000 497213.594, 103705.500 49..."
622,Verspr. h. ten W. van de Vecht,537,510,9.497207e-07,0.949721,"MULTIPOLYGON (((130413.773 483352.540, 130487...."
...,...,...,...,...,...,...
418,Noorderveld landelijk gebied,6068,0,0.000000e+00,0.000000,"POLYGON ((107780.453 504633.719, 108077.883 50..."
698,Wolfskamer,1900,0,0.000000e+00,0.000000,"POLYGON ((145194.647 480361.529, 145154.246 48..."
666,Waterhout,13801,0,0.000000e+00,0.000000,"POLYGON ((141821.797 485936.906, 141961.203 48..."
36,Beatrixpark,5964,0,0.000000e+00,0.000000,"POLYGON ((141167.094 488250.594, 141700.364 48..."


In [29]:
breaks = jenkspy.jenks_breaks(areagdf_without_outliers['resident_1'], nb_class=2)
print(breaks)

areagdf_without_outliers['class_sqkm'] = pd.cut(areagdf_without_outliers['resident_1'],
                        bins=breaks,
                        labels=['low_density', 'high_density'])

print('total number without 5 outliers:', len(areagdf_without_outliers))

areagdf_without_outliers.head()

[0.0, 4.48765894e-07, 2.222222222e-06]
total number without 5 outliers: 734


C:\Users\Gamelab\miniconda3\envs\py36\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,buurt,residentia,population,resident_1,resident_2,geometry,class_sqkm
0,'s-Graveland,24796,1230,4.960477e-08,0.049605,"POLYGON ((137875.406 474664.812, 137907.328 47...",low_density
1,'t Hoogt van 't Kruis,57655,850,1.474287e-08,0.014743,"POLYGON ((140412.094 469255.812, 140788.797 46...",low_density
2,'t Spil,7877,260,3.300749e-08,0.033007,"POLYGON ((131114.406 496077.594, 131557.094 49...",low_density
3,Aalsmeerderbrug,23500,530,2.255319e-08,0.022553,"POLYGON ((113074.797 477238.594, 113177.148 47...",low_density
4,Abbenes,39800,1010,2.537688e-08,0.025377,"POLYGON ((100000.000 472367.688, 99831.039 472...",low_density


In [35]:
## see the number of low density and high density values in the dataframe where outliers are removed 
areagdf_without_outliers['class_sqkm'].value_counts()
#areagdf_without_outliers.loc[areagdf_without_outliers['class_sqkm'].isna()]
areagdf_without_outliers.loc[areagdf_without_outliers['class_sqkm']=='high_density']

,buurt,residentia,population,resident_1,resident_2,geometry,class_sqkm
21,Atol e.o.,6159,3340,5.422958e-07,0.542296,"POLYGON ((162217.406 503399.094, 162389.406 50...",high_density
123,De Hoek e.o. Hoofddorp,153,130,8.496732e-07,0.849673,"POLYGON ((108798.703 479239.000, 109984.102 47...",high_density
129,De Kwakel-Noord,1513,1070,7.072042e-07,0.707204,"POLYGON ((114550.688 474908.281, 115422.203 47...",high_density
220,Hilversumse Meent landelijk gebied,18,40,2.222222e-06,2.222222,"POLYGON ((135744.953 477545.688, 135908.547 47...",high_density
229,Hoofddorp e.o.,47284,56130,1.187082e-06,1.187082,"POLYGON ((110960.203 483659.188, 110722.602 48...",high_density
230,Hoofddorp-Bornholm,159,100,6.289308e-07,0.628931,"POLYGON ((106272.797 480780.188, 106478.602 48...",high_density
257,IJ-eiland e.o.,108,220,2.037037e-06,2.037037,"MULTIPOLYGON (((126447.395 488087.616, 126138....",high_density
288,Keizer Karelpark-Oost,6782,4590,6.767915e-07,0.676792,"POLYGON ((118778.703 479410.000, 119638.898 47...",high_density
324,Landelijk gebied Almere-Haven,13,10,7.692308e-07,0.769231,"POLYGON ((141183.886 483066.422, 141170.299 48...",high_density
556,Sloterweg-Zuid Schiphol,323,230,7.120743e-07,0.712074,"POLYGON ((112441.602 483785.906, 113108.797 48...",high_density


In [36]:
## fisher jenks on the low-density alone 
areagdf_without_outliers_lowdensity = areagdf_without_outliers.loc[areagdf_without_outliers['class_sqkm']=='low_density']
areagdf_without_outliers_lowdensity.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 712 entries, 0 to 738
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   buurt       712 non-null    object  
 1   residentia  712 non-null    int64   
 2   population  712 non-null    int64   
 3   resident_1  712 non-null    float64 
 4   resident_2  712 non-null    float64 
 5   geometry    712 non-null    geometry
 6   class_sqkm  712 non-null    category
dtypes: category(1), float64(2), geometry(1), int64(2), object(1)
memory usage: 39.7+ KB


In [37]:
## jenks on low density:
breaks = jenkspy.jenks_breaks(areagdf_without_outliers_lowdensity['resident_1'], nb_class=2)
print(breaks)

areagdf_without_outliers_lowdensity['class_sqkm_ld'] = pd.cut(areagdf_without_outliers_lowdensity['resident_1'],
                        bins=breaks,
                        labels=['low_density', 'high_density'])
areagdf_without_outliers_lowdensity.info()

[7.1299e-10, 1.66986974e-07, 4.48765894e-07]
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 712 entries, 0 to 738
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   buurt          712 non-null    object  
 1   residentia     712 non-null    int64   
 2   population     712 non-null    int64   
 3   resident_1     712 non-null    float64 
 4   resident_2     712 non-null    float64 
 5   geometry       712 non-null    geometry
 6   class_sqkm     712 non-null    category
 7   class_sqkm_ld  711 non-null    category
dtypes: category(2), float64(2), geometry(1), int64(2), object(1)
memory usage: 40.5+ KB


C:\Users\Gamelab\miniconda3\envs\py36\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [39]:
areagdf_without_outliers_lowdensity['class_sqkm_ld'].value_counts()
areagdf_without_outliers_lowdensity.loc[areagdf_without_outliers_lowdensity['class_sqkm_ld']=='high_density']

,buurt,residentia,population,resident_1,resident_2,geometry,class_sqkm,class_sqkm_ld
46,Beukenhorst,1337,600,4.487659e-07,0.448766,"POLYGON ((125568.898 483539.188, 125921.000 48...",low_density,high_density
57,Binnengouw,4270,980,2.295082e-07,0.229508,"POLYGON ((131951.890 495920.286, 131945.860 49...",low_density,high_density
172,Egelshoek,328,130,3.963415e-07,0.396341,"POLYGON ((140421.521 465621.151, 140123.546 46...",low_density,high_density
314,Kronenburg en Uilenstede,7649,3170,4.144333e-07,0.414433,"POLYGON ((120495.797 481699.094, 120504.398 48...",low_density,high_density
333,Laren-Natuurgebied,1178,480,4.074703e-07,0.407470,"POLYGON ((144838.596 473079.657, 144837.917 47...",low_density,high_density
385,N.-Vennep e.o.,2674,810,3.029170e-07,0.302917,"POLYGON ((102531.797 478823.688, 103348.703 47...",low_density,high_density
495,Provinciaal Ziekenhuis,1642,410,2.496955e-07,0.249695,"POLYGON ((102480.289 493409.656, 102630.977 49...",low_density,high_density
580,Studentenflats,4205,920,2.187872e-07,0.218787,"POLYGON ((126740.797 483620.188, 126645.500 48...",low_density,high_density
624,Verspr. h. ten Z.W. van Middenbeemster,2323,450,1.937150e-07,0.193715,"POLYGON ((117577.797 506038.861, 118033.544 50...",low_density,high_density


In [40]:
areagdf_without_outliers_lowdensity2 = areagdf_without_outliers_lowdensity.loc[areagdf_without_outliers_lowdensity['class_sqkm_ld']=='low_density']
## jenks on low density:
breaks = jenkspy.jenks_breaks(areagdf_without_outliers_lowdensity2['resident_1'], nb_class=2)
print(breaks)

areagdf_without_outliers_lowdensity2['class_sqkm_ld2'] = pd.cut(areagdf_without_outliers_lowdensity2['resident_1'],
                        bins=breaks,
                        labels=['low_density', 'high_density'])
areagdf_without_outliers_lowdensity2.info()
areagdf_without_outliers_lowdensity2['class_sqkm_ld2'].value_counts()



[8.3039e-10, 4.7658076e-08, 1.66986974e-07]
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 702 entries, 0 to 738
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   buurt           702 non-null    object  
 1   residentia      702 non-null    int64   
 2   population      702 non-null    int64   
 3   resident_1      702 non-null    float64 
 4   resident_2      702 non-null    float64 
 5   geometry        702 non-null    geometry
 6   class_sqkm      702 non-null    category
 7   class_sqkm_ld   702 non-null    category
 8   class_sqkm_ld2  701 non-null    category
dtypes: category(3), float64(2), geometry(1), int64(2), object(1)
memory usage: 40.7+ KB


C:\Users\Gamelab\miniconda3\envs\py36\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


low_density     595
high_density    106
Name: class_sqkm_ld2, dtype: int64

In [41]:
areagdf_without_outliers_lowdensity2.loc[areagdf_without_outliers_lowdensity2['class_sqkm_ld2']=='high_density'].head(10)

,buurt,residentia,population,resident_1,resident_2,geometry,class_sqkm,class_sqkm_ld,class_sqkm_ld2
0,'s-Graveland,24796,1230,4.960477e-08,0.049605,"POLYGON ((137875.406 474664.812, 137907.328 47...",low_density,low_density,high_density
5,Achtersluispolder,2388,140,5.862647e-08,0.058626,"POLYGON ((118204.578 493861.281, 119141.609 49...",low_density,low_density,high_density
13,Ambachtskwartier Houtwegen,6312,350,5.544994e-08,0.055450,"POLYGON ((105219.602 502531.688, 105383.297 50...",low_density,low_density,high_density
43,Bellatrixbuurt,15200,1060,6.973684e-08,0.069737,"POLYGON ((100823.703 496439.500, 101241.203 49...",low_density,low_density,high_density
50,"Bijlmer-Centrum (D,F,H)",340480,22390,6.576010e-08,0.065760,"POLYGON ((124835.648 482224.375, 125040.172 48...",low_density,low_density,high_density
51,"Bijlmer-Oost (E,G,K)",450094,27160,6.034295e-08,0.060343,"POLYGON ((127107.141 482651.062, 127240.789 48...",low_density,low_density,high_density
61,Blijkpolder,17324,1020,5.887786e-08,0.058878,"POLYGON ((131738.594 474342.812, 131805.297 47...",low_density,low_density,high_density
84,Brink,20644,990,4.795582e-08,0.047956,"POLYGON ((139447.906 476400.812, 139838.297 47...",low_density,low_density,high_density
86,Buikslotermeer,177379,10420,5.874427e-08,0.058744,"POLYGON ((124800.102 491902.094, 124863.797 49...",low_density,low_density,high_density
88,Buitendijken Muiderberg,27251,1450,5.320906e-08,0.053209,"POLYGON ((136858.500 481773.188, 136773.406 48...",low_density,low_density,high_density


## First jenks without the 5 outliers 


## Fisher Jenks classification

[0.0, 3.02916978e-07, 1.187082311e-06]


,buurt,residentia,population,resident_1,resident_2,geometry,class_sqkm
0,'s-Graveland,24796,1230,4.960477e-08,0.049605,"POLYGON ((137875.406 474664.812, 137907.328 47...",low_density
1,'t Hoogt van 't Kruis,57655,850,1.474287e-08,0.014743,"POLYGON ((140412.094 469255.812, 140788.797 46...",low_density
2,'t Spil,7877,260,3.300749e-08,0.033007,"POLYGON ((131114.406 496077.594, 131557.094 49...",low_density
3,Aalsmeerderbrug,23500,530,2.255319e-08,0.022553,"POLYGON ((113074.797 477238.594, 113177.148 47...",low_density
4,Abbenes,39800,1010,2.537688e-08,0.025377,"POLYGON ((100000.000 472367.688, 99831.039 472...",low_density


In [7]:
## Low density 
areagdf.loc[areagdf['class_sqkm']=='low_density']

,buurt,residentia,population,resident_1,resident_2,geometry,class_sqkm
0,'s-Graveland,24796,1230,4.960477e-08,0.049605,"POLYGON ((137875.406 474664.812, 137907.328 47...",low_density
1,'t Hoogt van 't Kruis,57655,850,1.474287e-08,0.014743,"POLYGON ((140412.094 469255.812, 140788.797 46...",low_density
2,'t Spil,7877,260,3.300749e-08,0.033007,"POLYGON ((131114.406 496077.594, 131557.094 49...",low_density
3,Aalsmeerderbrug,23500,530,2.255319e-08,0.022553,"POLYGON ((113074.797 477238.594, 113177.148 47...",low_density
4,Abbenes,39800,1010,2.537688e-08,0.025377,"POLYGON ((100000.000 472367.688, 99831.039 472...",low_density
...,...,...,...,...,...,...,...
734,Zwanenburg-Noordwest,50003,1220,2.439854e-08,0.024399,"POLYGON ((111043.102 488352.406, 111261.102 48...",low_density
735,Zwanenburg-Oost,42756,1110,2.596127e-08,0.025961,"POLYGON ((111705.398 487646.500, 111957.203 48...",low_density
736,Zwanenburg-West,62514,2220,3.551204e-08,0.035512,"POLYGON ((109520.102 488557.500, 110133.797 48...",low_density
737,Zwanenburg-Zuidoost,33162,860,2.593330e-08,0.025933,"POLYGON ((111705.398 487646.500, 111416.297 48...",low_density


In [17]:
# In square meters:
breaks = jenkspy.jenks_breaks(areagdf_without_outliers['resident_2'], nb_class=2)
print(breaks)
areagdf['class_sqm'] = pd.cut(areagdf_without_outliers['resident_2'],
                        bins=breaks,
                        labels=['low_density', 'high_density'])
areagdf.head()

[0.0, 0.302916978309648, 1.187082311141189]


,buurt,residentia,population,resident_1,resident_2,geometry,class_sqkm,class_sqm
0,'s-Graveland,24796,1230,4.960477e-08,0.049605,"POLYGON ((137875.406 474664.812, 137907.328 47...",low_density,low_density
1,'t Hoogt van 't Kruis,57655,850,1.474287e-08,0.014743,"POLYGON ((140412.094 469255.812, 140788.797 46...",low_density,low_density
2,'t Spil,7877,260,3.300749e-08,0.033007,"POLYGON ((131114.406 496077.594, 131557.094 49...",low_density,low_density
3,Aalsmeerderbrug,23500,530,2.255319e-08,0.022553,"POLYGON ((113074.797 477238.594, 113177.148 47...",low_density,low_density
4,Abbenes,39800,1010,2.537688e-08,0.025377,"POLYGON ((100000.000 472367.688, 99831.039 472...",low_density,low_density


In [20]:
areagdf['class_sqkm'].value_counts()
areagdf.loc[areagdf['class_sqkm']=='high_density'].to_csv(path+'\\high_density_buurts_after_outliers.csv')

Removing outliers:  [the high rise cases] the natural breaks in square meters are: 
- low density:      0.0     to      0.448 sq meters
- high density:     0.448   to      2.037 sq meters

## Quantile classification

In [6]:
areadf['quantile_sqkm'] = pd.qcut(areadf['residentialdensity_sqkm'], q=2, labels=['low_density', 'high_density'])
areadf['quantile_sqm'] = pd.qcut(areagdf['residential_sqm'], q=2, labels=['low_density', 'high_density'])

areadf['quantile_sqkm'].value_counts(),areadf['quantile_sqm'].value_counts()

(low_density     370
 high_density    369
 Name: quantile_sqkm, dtype: int64,
 low_density     370
 high_density    369
 Name: quantile_sqm, dtype: int64)

In [8]:
areadf.loc[areadf['quantile_sqm']=='low_density', 'residential_sqm'].min(), areadf.loc[areadf['quantile_sqm']=='low_density', 'residential_sqm'].max()

(0.0, 0.027854097584083373)

In [9]:
areadf.loc[areadf['quantile_sqm']=='high_density', 'residential_sqm'].min(), areadf.loc[areadf['quantile_sqm']=='high_density', 'residential_sqm'].max()

(0.027889805172655952, 50.46511627906977)

# 2. Residential Density - Method 02 

Residential Density = (Population of buurt / residential area of that buurt)

residential area of buurt = sum(areas of residential patches)

In [31]:
residential_patches = gpd.read_file('C:\\Users\\Gamelab\\Desktop\\RT\\Resilience_Lab\\MRA_COMPILED_2\\population_density\\mra_popdensity\\1996_residential_patches_with_buurts_FINAL_1.shp')
residential_patches = residential_patches.drop(columns=['statnaam','residentia','resident_1','resident_2','path', 'buurt'])
residential_patches.to_file('1996_residential_patches_split_area_only.shp')  ## this is to recalculate buurts after the intersection, split and merge. 


In [25]:
residential_patches = gpd.read_file('C:\\Users\\Gamelab\\Desktop\\RT\\Resilience_Lab\\MRA_COMPILED_2\\population_density\\mra_popdensity\\mra_intersection_correct.shp')
residential_patches = residential_patches.drop(columns=['Shape_Leng','Shape_Area','population','Inwoners','density_cl','wordt1996'])
#residential_patches = residential_patches.rename(columns={'statnaam':'buurt'})
residential_patches = residential_patches.rename(columns={'C28992R500':'box'})
residential_patches = residential_patches.loc[residential_patches['INW2000']>0]
buurts_resarea_dict= residential_patches.groupby('box')['area'].sum().to_dict()

buurts_pop_dict = residential_patches.groupby('box')['INW2000'].median().to_dict()

#calculate residential density of each buurt
buurts_residential_density = pd.DataFrame()
buurts_residential_density['box'] = list(buurts_pop_dict.keys())
buurts_residential_density['res_area'] = buurts_residential_density['box'].map(buurts_resarea_dict)
buurts_residential_density['population']= buurts_residential_density['box'].map(buurts_pop_dict)

#res density in square meters
buurts_residential_density['res_dens'] = buurts_residential_density['population']/buurts_residential_density['res_area']
buurts_residential_density = buurts_residential_density.loc[buurts_residential_density['res_dens']<1.25]
#edge_cases = ['Assendelft-Noord','Lisse','Nigtevecht','Oud-Over','Leimuiden']
#buurts_residential_density = buurts_residential_density.loc[~buurts_residential_density['buurts'].isin(edge_cases)]
#jenks classification
breaks = jenkspy.jenks_breaks(buurts_residential_density['res_dens'], nb_class=11)
print('jenks breaks:', breaks)
buurts_residential_density['dens_class'] = pd.cut(buurts_residential_density['res_dens'],
                      bins=breaks,
                      labels=['cluster1', 'cluster2','cluster3','cluster4','cluster5','cluster6','cluster7','cluster8','cluster9','cluster10','cluster11'])

#buurts_residential_density['dens_class'] = pd.qcut(buurts_residential_density['res_dens'], q=2, labels=['low_density', 'high_density'])
buurts_residential_density=buurts_residential_density.dropna(subset=['dens_class'])
buurts_residential_density['dens_class']= buurts_residential_density['dens_class'].astype('string')

classification = {'cluster1':'low_density','cluster2':'medium_density','cluster3':'high_density','cluster4':'high_density','cluster5':'high_density','cluster6':'high_density',\
    'cluster7':'high_density','cluster8':'high_density','cluster9':'high_density','cluster10':'high_density','cluster11':'high_density'}


buurts_residential_density['class'] = buurts_residential_density['dens_class'].map(classification)

print(buurts_residential_density['class'].value_counts())

##attach low density, high density to the residential patches 
buurts_residential_density = buurts_residential_density.set_index('box')
buurts_densities_dict = buurts_residential_density['class'].to_dict()
buurts_resdens_dict = buurts_residential_density['res_dens'].to_dict()

residential_patches['class']= residential_patches['box'].map(buurts_densities_dict)
residential_patches['res_dens']= residential_patches['box'].map(buurts_resdens_dict)



ld= residential_patches.loc[residential_patches['class']=='low_density']
wordt96 = [85 for i in range(len(ld))]
ld['wordt1996'] = wordt96


md=residential_patches.loc[residential_patches['class']=='medium_density']
wordt96 = [86 for i in range(len(md))]
md['wordt1996'] = wordt96

hd = residential_patches.loc[residential_patches['class']=='high_density']
wordt96 = [87 for i in range(len(hd))]
hd['wordt1996'] = wordt96


ld.to_file('1996_residential_patches_lowdensity.shp')
md.to_file('1996_residential_patches_mediumdensity.shp')
hd.to_file('1996_residential_patches_highdensity.shp')


residential_patches.to_file('1996_residential_patches_all_labelled.shp')


jenks breaks: [4.897455445399086e-06, 0.0024312279603900112, 0.007590132827324478, 0.025341630846218002, 0.09036144578313253, 0.15293637846655792, 0.23377529658060014, 0.47047047047047047, 0.5384615384615384, 0.6063268892794376, 0.8155487804878049, 0.8917682926829268]
low_density       1799
medium_density     440
high_density       100
Name: class, dtype: int64


C:\Users\Gamelab\miniconda3\envs\py36\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [18]:

buurts_residential_density['dens_class'].value_counts()

cluster1     1799
cluster2      440
cluster3       69
cluster4       16
cluster6        5
cluster5        4
cluster7        2
cluster11       1
cluster9        1
cluster10       1
cluster8        1
Name: dens_class, dtype: Int64

In [95]:
## jenks2 
##get low density 
buurts_residential_density = buurts_residential_density.loc[buurts_residential_density['dens_class']=='low_density']
breaks = jenkspy.jenks_breaks(buurts_residential_density['res_dens'], nb_class=2)
print('jenks breaks:', breaks)
buurts_residential_density['dens_class2'] = pd.cut(buurts_residential_density['res_dens'],
                       bins=breaks,
                       labels=['low_density', 'high_density'])
print(buurts_residential_density['dens_class2'].value_counts())

jenks breaks: [5.317041916899952e-06, 0.07040816326530612, 0.23377529658060014]
low_density     2322
high_density      10
Name: dens_class2, dtype: int64


In [96]:
## jenks3 
buurts_residential_density = buurts_residential_density.loc[buurts_residential_density['dens_class2']=='low_density']
breaks = jenkspy.jenks_breaks(buurts_residential_density['res_dens'], nb_class=2)
print('jenks breaks:', breaks)
buurts_residential_density['dens_class3'] = pd.cut(buurts_residential_density['res_dens'],
                       bins=breaks,
                       labels=['low_density', 'high_density'])

print(buurts_residential_density['dens_class3'].value_counts())
                       

jenks breaks: [5.6256448395397325e-06, 0.025341630846218002, 0.07040816326530612]
low_density     2306
high_density      15
Name: dens_class3, dtype: int64


In [97]:
## jenks4 
buurts_residential_density = buurts_residential_density.loc[buurts_residential_density['dens_class3']=='low_density']
breaks = jenkspy.jenks_breaks(buurts_residential_density['res_dens'], nb_class=2)
print('jenks breaks:', breaks)
buurts_residential_density['dens_class4'] = pd.cut(buurts_residential_density['res_dens'],
                       bins=breaks,
                       labels=['low_density', 'high_density'])

print(buurts_residential_density['dens_class4'].value_counts())
                       

jenks breaks: [6.405083073927469e-06, 0.00421591623357709, 0.025341630846218002]
low_density     2076
high_density     229
Name: dens_class4, dtype: int64


In [93]:
## QUANTILE 

#buurts_residential_density['res_dens'].sort_values(ascending= False)[:10]
buurts_residential_density.loc[buurts_residential_density['dens_class']=='low_density','res_dens'].sort_values(ascending=False)
#buurts_residential_density.loc[buurts_residential_density['dens_class']=='high_density','res_dens'].sort_values(ascending=False)

box
E1600N5010    0.001034
E1085N5050    0.001033
E1145N5005    0.001032
E1345N4680    0.001030
E1165N4760    0.001024
                ...   
E1185N4835    0.000007
E1015N4915    0.000006
E1170N4760    0.000006
E1015N4900    0.000005
E1370N4680    0.000005
Name: res_dens, Length: 1170, dtype: float64

In [71]:
buurts_residential_density.loc[buurts_residential_density.index=='E1430N4750']

,res_area,population,res_dens,dens_class
box,,,,
E1430N4750,1,220,220.0,high_density


In [42]:
buurts_residential_density.loc[buurts_residential_density['dens_class']=='high_density']

,res_area,population,res_dens,dens_class
buurts,,,,
Regenboogbuurt,739,2500,3.382950,high_density
Vlindertuin,603,1450,2.404643,high_density


In [44]:
buurts_residential_density = buurts_residential_density.loc[buurts_residential_density['dens_class']=='low_density']

breaks = jenkspy.jenks_breaks(buurts_residential_density['res_dens'], nb_class=2)
print('jenks breaks:', breaks)
buurts_residential_density['dens_class2'] = pd.cut(buurts_residential_density['res_dens'],
                        bins=breaks,
                        labels=['low_density', 'high_density'])

print(buurts_residential_density['dens_class2'].value_counts())

jenks breaks: [1.6419687204958747e-05, 0.4641350210970464, 1.1299245599329422]
low_density     763
high_density      2
Name: dens_class2, dtype: int64


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7933 entries, 0 to 7932
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   fid            7933 non-null   float64 
 1   OBJECTID       7933 non-null   float64 
 2   wordt1996      7933 non-null   int64   
 3   Shape_Leng     7933 non-null   float64 
 4   Shape_Area     7933 non-null   float64 
 5   population     6758 non-null   float64 
 6   buurt          4836 non-null   object  
 7   geometry       7933 non-null   geometry
 8   density_class  4698 non-null   object  
dtypes: float64(5), geometry(1), int64(1), object(2)
memory usage: 557.9+ KB


# 2006

In [62]:
respatches2006 = gpd.read_file(rootpath+'mra_popdensity\\2006\\2006_respatches_populationvalues_area.shp')
popboxes = gpd.read_file(rootpath+'mra_popdensity\\mra_population_500X500.shp')
popboxes= popboxes.set_index('C28992R500')

new_fid= [i+1 for i in range(len(respatches2006))]

respatches2006['newid']= new_fid
respatches2006 = respatches2006.set_index('newid')
respatches_geomap = respatches2006['geometry'].to_dict()
respatches2006.head(2)

,fid,OBJECTID,wordt2006,Shape_Leng,Shape_Area,C28992R500,INW2000,INW2001,INW2002,INW2003,...,STED2010,STED2011,STED2012,STED2013,WON1971,INW1971,OAD2014,STED2014,area,geometry
newid,,,,,,,,,,,,,,,,,,,,,
1,39.0,296625.0,5,50.620341,3.265556,E1005N4720,290,285,265,255,...,5,5,5,5,70,265,127,5,3,"POLYGON ((100684.646 472165.500, 100668.703 47..."
2,40.0,296626.0,5,97.367587,9.587680,E1005N4720,290,285,265,255,...,5,5,5,5,70,265,127,5,10,"POLYGON ((100617.215 472113.203, 100657.736 47..."


Jenks 11

Low density 1799 (4.89- 06 to 0.002)

Medium density 440 (0.002 - 0.007)

High density 100 (0.007 - 0.89)

In [64]:
#group by boxes and sum up the res areas within each box
resdens_df = pd.DataFrame()
resareas = respatches2006.groupby('C28992R500')['area'].sum().to_dict()
populations = respatches2006.groupby('C28992R500')['INW2006'].median().to_dict()

resdens_df['boxes']= list(resareas.keys())
resdens_df['res_area']= resdens_df['boxes'].map(resareas)
resdens_df['population']= resdens_df['boxes'].map(populations)
resdens_df['res_density']= round(resdens_df['population']/resdens_df['res_area'],3)
resdens_df = resdens_df.loc[resdens_df['res_density']>=0]

classes=[]

# for _,row in resdens_df.iterrows():
#     if row['res_density']<= 0.002:
#         classes.append('low_density')
#     if (row['res_density']>=0.0020001) & (row['res_density']<=0.007):
#         classes.append('medium_density')
#     if (row['res_density']>=0.0070001) & (row['res_density']<=0.9):
#         classes.append('high_density')
#     else:
#         classes.append('low_density')
        
# resdens_df['classes']= classes
#resdens_df


#bool_series = resdens_df["res_density"].between(-200, 0.002, inclusive = True)
#low_res = resdens_df[bool_series]
#classes = ['low_density' for i in range(len(low_res))]

#low_res['classes']= classes

from geopandas import GeoDataFrame
from shapely.geometry import Point

#lowres_gdf = GeoDataFrame(low_res, crs="EPSG:28992", geometry=low_res['geometry'])

#resdens_gdf = GeoDataFrame(resdens_df, crs="EPSG:28992", geometry=resdens_df['geometry'])

#lowres_gdf.to_file(rootpath+'mra_popdensity\\2006\\popboxes_lowres.shp')


In [43]:
bool_series = resdens_df["res_density"].between(0.0021, 0.007, inclusive = True)
med_res = resdens_df[bool_series]

classes = ['medium_density' for i in range(len(med_res))]

med_res['classes']= classes

from geopandas import GeoDataFrame
from shapely.geometry import Point

medres_gdf = GeoDataFrame(med_res, crs="EPSG:28992", geometry=med_res['geometry'])

#medres_gdf.to_file(rootpath+'mra_popdensity\\2006\\popboxes_medres.shp')


C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  del sys.path[0]


In [44]:
bool_series = resdens_df["res_density"].between(0.007000001, 0.9, inclusive = True)
highres = resdens_df[bool_series]


classes = ['high_density' for i in range(len(highres))]

highres['classes']= classes

from geopandas import GeoDataFrame
from shapely.geometry import Point

highres_gdf = GeoDataFrame(highres, crs="EPSG:28992", geometry=highres['geometry'])

#highres_gdf.to_file(rootpath+'mra_popdensity\\2006\\popboxes_highres.shp')

C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:14: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


In [68]:
respatches2006.columns

Index(['fid', 'OBJECTID', 'wordt2006', 'Shape_Leng', 'Shape_Area',
       'C28992R500', 'INW2000', 'INW2001', 'INW2002', 'INW2003',
       ...
       'STED2010', 'STED2011', 'STED2012', 'STED2013', 'WON1971', 'INW1971',
       'OAD2014', 'STED2014', 'area', 'geometry'],
      dtype='object', length=115)

In [53]:
popboxes_all = gpd.read_file(rootpath+'mra_popdensity\\2006\\popboxes_allres.shp')
popboxes_all = popboxes_all.set_index('boxes')
box_class = popboxes_all['classes'].to_dict()

In [69]:
resdens_df['classes']= resdens_df['boxes'].map(box_class)
resdens_df = resdens_df.set_index('boxes')
box_density = resdens_df['res_density'].to_dict()

respatches2006['classes']= respatches2006['C28992R500'].map(box_class)
respatches2006['density']= respatches2006['C28992R500'].map(box_density)
respatches2006 = respatches2006[['fid','OBJECTID','wordt2006','area','INW2006','density','classes','geometry']]
respatches2006= respatches2006.dropna(subset=['density'])
respatches2006= respatches2006.dropna(subset=['classes'])
respatches2006.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 11503 entries, 1 to 11503
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   fid        11503 non-null  float64 
 1   OBJECTID   11503 non-null  float64 
 2   wordt2006  11503 non-null  int64   
 3   area       11503 non-null  int64   
 4   INW2006    11503 non-null  int64   
 5   density    11413 non-null  float64 
 6   classes    11399 non-null  object  
 7   geometry   11503 non-null  geometry
dtypes: float64(3), geometry(1), int64(3), object(1)
memory usage: 808.8+ KB


In [79]:
ld = respatches2006.loc[respatches2006['classes']=='low_density']
wordt = [85 for i in range(len(ld))]
ld['wordt2006_']= wordt
ld = ld.drop(columns='wordt2006')
ld = ld.rename(columns={'wordt2006_':'wordt2006'})


md = respatches2006.loc[respatches2006['classes']=='medium_density']
wordt = [86 for i in range(len(md))]
md['wordt2006_']= wordt
md = md.drop(columns='wordt2006')
md = md.rename(columns={'wordt2006_':'wordt2006'})


hd = respatches2006.loc[respatches2006['classes']=='high_density']
wordt = [87 for i in range(len(hd))]
hd['wordt2006_']= wordt
hd = hd.drop(columns='wordt2006')
hd = hd.rename(columns={'wordt2006_':'wordt2006'})

ld.info(),md.info(),hd.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 671 entries, 325 to 11501
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   fid        671 non-null    float64 
 1   OBJECTID   671 non-null    float64 
 2   area       671 non-null    int64   
 3   INW2006    671 non-null    int64   
 4   density    671 non-null    float64 
 5   classes    671 non-null    object  
 6   geometry   671 non-null    geometry
 7   wordt2006  671 non-null    int64   
dtypes: float64(3), geometry(1), int64(3), object(1)
memory usage: 47.2+ KB
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4102 entries, 1 to 11503
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   fid        4102 non-null   float64 
 1   OBJECTID   4102 non-null   float64 
 2   area       4102 non-null   int64   
 3   INW2006    4102 non-null   int64   
 4   density    4102 non-null   float

C:\Users\Gamelab\miniconda3\envs\py36\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


(None, None, None)

In [80]:
ld.head()

,fid,OBJECTID,area,INW2006,density,classes,geometry,wordt2006
newid,,,,,,,,
325,430.0,298170.0,448,0,0.000,low_density,"POLYGON ((99484.872 481500.000, 99486.123 4815...",85
352,439.0,298179.0,3694,40,0.002,low_density,"POLYGON ((100665.640 482561.846, 100666.721 48...",85
358,441.0,298181.0,15969,40,0.002,low_density,"POLYGON ((100746.085 482500.000, 100747.284 48...",85
390,453.0,298193.0,1503,10,0.002,low_density,"POLYGON ((104530.560 483715.656, 104530.035 48...",85
548,1403.0,301093.0,2979,30,0.002,low_density,"POLYGON ((102959.234 477619.382, 102958.006 47...",85


In [81]:
ld.to_file(rootpath+'mra_popdensity\\2006\\2006_lowdensity_respatches.shp')
md.to_file(rootpath+'mra_popdensity\\2006\\2006_mediumdensity_respatches.shp')
hd.to_file(rootpath+'mra_popdensity\\2006\\2006_highdensity_respatches.shp')

# 2015

In [4]:
respatches2015 = gpd.read_file(path+'\\2015\\respatches2015_popbox_area.shp')
respatches2015

,id,C28992R500,INWONER,area,geometry
0,None,E1005N4820,1225,1334,"POLYGON ((4.58785 52.32459, 4.58786 52.32460, ..."
1,None,E1005N4825,45,308,"POLYGON ((4.58978 52.32775, 4.58999 52.32811, ..."
2,None,E1005N4820,1225,673,"POLYGON ((4.58911 52.32661, 4.58911 52.32661, ..."
3,None,E1005N4720,260,5,"POLYGON ((4.59066 52.23338, 4.59068 52.23339, ..."
4,None,E1005N4715,290,26,"POLYGON ((4.59075 52.23335, 4.59073 52.23334, ..."
...,...,...,...,...,...
13319,None,E1595N5050,115,139,"POLYGON ((5.45740 52.53605, 5.45726 52.53585, ..."
13320,None,E1600N5060,190,255,"POLYGON ((5.46212 52.54354, 5.46197 52.54337, ..."
13321,None,E1590N5055,25,447,"POLYGON ((5.45353 52.53931, 5.45331 52.53896, ..."
13322,None,E1595N5055,210,295,"POLYGON ((5.45410 52.54016, 5.45462 52.54003, ..."


In [5]:
#respatches contain the geometries of the patches, to which you have to attach a density class, the residential density
  
box_resarea = respatches2015.groupby('C28992R500')['area'].sum().to_dict()  #res area of each box
box_pop = respatches2015.groupby('C28992R500')['INWONER'].median().to_dict()  #population of each box

#give id to respatches
ids = [i for i in range(len(respatches2015))]
respatches2015['fid']= ids
respatches2015 = respatches2015.drop(columns=['id'])

# calculate res density 
boxdf = pd.DataFrame()
boxdf['box'] = list(box_resarea.keys())
boxdf['resarea']= boxdf['box'].map(box_resarea)
boxdf['population']=boxdf['box'].map(box_pop)
boxdf['res_density']= boxdf['population']/boxdf['resarea']

boxdf = boxdf.loc[boxdf['res_density']>=0]

#classify into
lowdens = boxdf["res_density"].between(-200, 0.002, inclusive = True)
lowdens_df = boxdf[lowdens]
classes = ['low_density' for i in range(len(lowdens_df))]
lowdens_df['classes']=classes

med_dens = boxdf["res_density"].between(0.00205, 0.007, inclusive = True)
meddens_df = boxdf[med_dens]
classes = ['medium_density' for i in range(len(meddens_df))]
meddens_df['classes']=classes

highdens = boxdf["res_density"].between(0.00705, 2, inclusive = True)
highdens_df = boxdf[highdens]
classes = ['high_density' for i in range(len(highdens_df))]
highdens_df['classes']=classes


boxes_all = pd.concat([lowdens_df,meddens_df])
boxes_all = pd.concat([boxes_all,highdens_df])

boxes_all



C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,box,resarea,population,res_density,classes
38,E0990N4860,53892,90,0.001670,low_density
46,E0995N4860,103928,180,0.001732,low_density
48,E0995N4870,90808,50,0.000551,low_density
59,E1000N4850,5221,5,0.000958,low_density
60,E1000N4855,33687,35,0.001039,low_density
...,...,...,...,...,...
2612,E1620N5005,206380,1930,0.009352,high_density
2614,E1620N5015,223726,1850,0.008269,high_density
2626,E1625N5015,62157,505,0.008125,high_density
2637,E1630N5020,68970,540,0.007829,high_density


In [7]:
boxes_all = boxes_all.set_index('box')
box_class= boxes_all['classes'].to_dict()
box_density = boxes_all['res_density'].to_dict()

#add to residenitla patches 

respatches2015['res_density']= respatches2015['C28992R500'].map(box_density)
respatches2015['classes']= respatches2015['C28992R500'].map(box_class)

respatches2015.info()



<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13324 entries, 0 to 13323
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   C28992R500   13324 non-null  object  
 1   INWONER      13324 non-null  int64   
 2   area         13324 non-null  int64   
 3   geometry     13324 non-null  geometry
 4   fid          13324 non-null  int64   
 5   res_density  13087 non-null  float64 
 6   classes      13087 non-null  object  
dtypes: float64(1), geometry(1), int64(3), object(2)
memory usage: 728.8+ KB


In [11]:
respatches2015 = respatches2015.loc[~respatches2015['classes'].isna()]

ld = respatches2015.loc[respatches2015['classes']=='low_density']
wordt = [85 for i in range(len(ld))]
ld['wordt2015']= wordt



md = respatches2015.loc[respatches2015['classes']=='medium_density']
wordt = [86 for i in range(len(md))]
md['wordt2015']= wordt



hd = respatches2015.loc[respatches2015['classes']=='high_density']
wordt = [87 for i in range(len(hd))]
hd['wordt2015']= wordt


ld.info(),md.info(),hd.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 454 entries, 351 to 13322
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   C28992R500   454 non-null    object  
 1   INWONER      454 non-null    int64   
 2   area         454 non-null    int64   
 3   geometry     454 non-null    geometry
 4   fid          454 non-null    int64   
 5   res_density  454 non-null    float64 
 6   classes      454 non-null    object  
 7   wordt2015    454 non-null    int64   
dtypes: float64(1), geometry(1), int64(4), object(2)
memory usage: 31.9+ KB
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4167 entries, 0 to 13323
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   C28992R500   4167 non-null   object  
 1   INWONER      4167 non-null   int64   
 2   area         4167 non-null   int64   
 3   geometry     4167 non-null   geometry
 4   

(None, None, None)

In [12]:
ld.to_file(rootpath+'mra_popdensity\\2015\\2015_lowdensity_respatches.shp')
md.to_file(rootpath+'mra_popdensity\\2015\\2015_mediumdensity_respatches.shp')
hd.to_file(rootpath+'mra_popdensity\\2015\\2015_highdensity_respatches.shp')

C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  
C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


# 2006 100by100

In [6]:
respatches2006 = gpd.read_file(path+'\\2006\\100by100\\2006_respatches_100by100_pop_area.shp')
respatches2006 = respatches2006.loc[respatches2006['INW2006']>=0]
respatches2006

,fid,OBJECTID,wordt2006,Shape_Leng,Shape_Area,C28992R100,INW2006,area,geometry
0,39.0,296625.0,5,50.620341,3.265556,E1006N4721,10,3,"POLYGON ((100684.646 472165.500, 100668.703 47..."
1,40.0,296626.0,5,97.367587,9.587680,E1006N4721,10,10,"POLYGON ((100617.215 472113.203, 100657.736 47..."
2,95.0,296791.0,5,367.277837,1228.995747,E1047N4709,10,846,"POLYGON ((104800.000 470977.463, 104791.842 47..."
4,151.0,297033.0,5,1019.994822,7102.477182,E1037N4757,25,221,"POLYGON ((103800.000 475700.888, 103799.204 47..."
5,151.0,297033.0,5,1019.994822,7102.477182,E1036N4755,30,1907,"POLYGON ((103700.000 475590.219, 103669.859 47..."
...,...,...,...,...,...,...,...,...,...
49233,55654.0,675768.0,5,827.023806,747.809632,E1595N5058,0,142,"POLYGON ((159538.066 505837.001, 159573.354 50..."
49234,55654.0,675768.0,5,827.023806,747.809632,E1594N5055,20,152,"POLYGON ((159444.046 505600.000, 159421.359 50..."
49235,55654.0,675768.0,5,827.023806,747.809632,E1595N5057,15,153,"POLYGON ((159522.060 505800.000, 159500.000 50..."
49236,55654.0,675768.0,5,827.023806,747.809632,E1594N5054,10,5,"POLYGON ((159405.740 505500.000, 159401.271 50..."


In [8]:
#respatches contain the geometries of the patches, to which you have to attach a density class, the residential density
  
box_resarea = respatches2006.groupby('C28992R100')['area'].sum().to_dict()  #res area of each box
box_pop = respatches2006.groupby('C28992R100')['INW2006'].median().to_dict()  #population of each box

#give id to respatches
#ids = [i for i in range(len(respatches2006))]
#respatches2006['fid']= ids
#respatches2015 = respatches2015.drop(columns=['id'])

# calculate res density 
boxdf = pd.DataFrame()
boxdf['box'] = list(box_resarea.keys())
boxdf['resarea']= boxdf['box'].map(box_resarea)
boxdf['population']=boxdf['box'].map(box_pop)
boxdf['res_density']= round(boxdf['population']/boxdf['resarea'],3)

boxdf = boxdf.loc[boxdf['res_density']>=0]


#classify into
lowdens = boxdf["res_density"].between(-200, 0.002, inclusive = True)
lowdens_df = boxdf[lowdens]
classes = ['low_density' for i in range(len(lowdens_df))]
lowdens_df['classes']=classes

med_dens = boxdf["res_density"].between(0.00205, 0.007, inclusive = True)
meddens_df = boxdf[med_dens]
classes = ['medium_density' for i in range(len(meddens_df))]
meddens_df['classes']=classes

highdens = boxdf["res_density"].between(0.00705, 2, inclusive = True)
highdens_df = boxdf[highdens]
classes = ['high_density' for i in range(len(highdens_df))]
highdens_df['classes']=classes


boxes_all = pd.concat([lowdens_df,meddens_df])
boxes_all = pd.concat([boxes_all,highdens_df])

boxes_all



C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,box,resarea,population,res_density,classes
2,E0958N4865,2769,5,0.002,low_density
6,E0959N4868,779,0,0.000,low_density
9,E0960N4869,6081,15,0.002,low_density
86,E0968N4871,10004,5,0.000,low_density
89,E0968N4874,8772,0,0.000,low_density
...,...,...,...,...,...
30683,E1632N5031,10004,80,0.008,high_density
30689,E1633N5023,4454,35,0.008,high_density
30690,E1633N5024,3869,45,0.012,high_density
30701,E1635N5004,437,5,0.011,high_density


In [9]:
boxes_all = boxes_all.set_index('box')
box_class= boxes_all['classes'].to_dict()
box_density = boxes_all['res_density'].to_dict()

#add to residenitla patches 

respatches2006['res_density']= respatches2006['C28992R100'].map(box_density)
respatches2006['classes']= respatches2006['C28992R100'].map(box_class)

respatches2006.info()



<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 47959 entries, 0 to 49237
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   fid          47959 non-null  float64 
 1   OBJECTID     47959 non-null  float64 
 2   wordt2006    47959 non-null  int64   
 3   Shape_Leng   47959 non-null  float64 
 4   Shape_Area   47959 non-null  float64 
 5   C28992R100   47959 non-null  object  
 6   INW2006      47959 non-null  int64   
 7   area         47959 non-null  int64   
 8   geometry     47959 non-null  geometry
 9   res_density  47851 non-null  float64 
 10  classes      47851 non-null  object  
dtypes: float64(5), geometry(1), int64(3), object(2)
memory usage: 4.4+ MB


In [10]:
respatches2006 = respatches2006.loc[~respatches2006['classes'].isna()]

ld = respatches2006.loc[respatches2006['classes']=='low_density']
wordt = [85 for i in range(len(ld))]
ld['wordt2006']= wordt



md = respatches2006.loc[respatches2006['classes']=='medium_density']
wordt = [86 for i in range(len(md))]
md['wordt2006']= wordt



hd = respatches2006.loc[respatches2006['classes']=='high_density']
wordt = [87 for i in range(len(hd))]
hd['wordt2006']= wordt


ld.info(),md.info(),hd.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6622 entries, 2 to 49237
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   fid          6622 non-null   float64 
 1   OBJECTID     6622 non-null   float64 
 2   wordt2006    6622 non-null   int64   
 3   Shape_Leng   6622 non-null   float64 
 4   Shape_Area   6622 non-null   float64 
 5   C28992R100   6622 non-null   object  
 6   INW2006      6622 non-null   int64   
 7   area         6622 non-null   int64   
 8   geometry     6622 non-null   geometry
 9   res_density  6622 non-null   float64 
 10  classes      6622 non-null   object  
dtypes: float64(5), geometry(1), int64(3), object(2)
memory usage: 620.8+ KB
<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 16860 entries, 5 to 49234
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   fid          16860 non-null  float64 
 1

C:\Users\Gamelab\miniconda3\envs\py36\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


(None, None, None)

In [11]:
ld.to_file(rootpath+'mra_popdensity\\2006\\100by100\\2006_100by100_lowdensity_respatches.shp')
md.to_file(rootpath+'mra_popdensity\\2006\\100by100\\2006_100by100_mediumdensity_respatches.shp')
hd.to_file(rootpath+'mra_popdensity\\2006\\100by100\\2006_100by100_highdensity_respatches.shp')

C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  
C:\Users\Gamelab\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until
